In [2]:
# BROUILLON 

**Import des bibliothèques nécessaires**

In [1]:
import pandas as pd
import spacy

**Machine learning : Import du fichier généré pour effectuer les tests**

In [36]:
df = pd.read_csv("../data/nlp_generated_text.csv")

In [37]:
df.head()

,texte,sujet
0,"Je suis blessé, j’ai besoin d’un centre médica...",santé
1,Je cherche un centre de santé à Paris où je po...,santé
2,"J’ai des pensées suicidaires, je suis à avenue...",santé
3,Y a-t-il un hôpital ou un médecin proche de ru...,santé
4,Je me suis blessé et je saigne. Où aller à Par...,santé


**Machine learning : Nettoyage du fichier**

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   texte   2000 non-null   object
 1   sujet   2000 non-null   object
dtypes: object(2)
memory usage: 31.4+ KB


In [39]:
# On vérifie si la colonne contient des doublons et on les traite s'il y en a.

filtered_df = df[df['texte'].isna()]
print(filtered_df)

df = df.dropna()

df.info()



Empty DataFrame
Columns: [texte, sujet]
Index: []
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   texte   2000 non-null   object
 1   sujet   2000 non-null   object
dtypes: object(2)
memory usage: 31.4+ KB


In [40]:
# On vérifie qu'on a bien que des valeurs avec un type String

print(df['texte'].apply(type).value_counts()) 

texte
<class 'str'>    2000
Name: count, dtype: int64


In [ ]:
# Réaliser le / les modèles de machine learning
# Le but : identifier le sujet concerné par le texte envoyé par l'utilisateur.
# Ce sujet identifié nous permettra de cibler la bonne table à requêter + nous servira aussi dans la partie NLP pour nommé ce sujet.

**API Chatbot**

**NLP : Chargement du modèle français de spaCy**

In [2]:
nlp = spacy.load("fr_core_news_sm")  # Chargement du modèle français


**NLP : Réception d'un texte en entrée**

In [51]:
# Etant dans un notebook Jupyter, je ne peux pas utiliser un input() pour le moment. Il faudra attendre de transférer le code dans un fichier python .py
# received_text = input(Bonjour ! Que puis-je faire pour vous ?")

received_text = "Je cherche les horaires d’une maraude dans Paris, si possible près de rue de Robin."
print ("vous : ", received_text)


vous :  Je cherche les horaires d’une maraude dans Paris, si possible près de rue de Robin.


In [3]:
# On récupère par web scrapping l a liste des termes se référentant au médical

import requests
from bs4 import BeautifulSoup
response = requests.get('https://www.hopital.fr/Le-dico-medical')


In [4]:
response

<Response [200]>

In [5]:
soup = BeautifulSoup ( response.content , "html.parser" )

In [6]:
soup.prettify()

'<!DOCTYPE html>\n<html>\n <head>\n  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>\n  <meta content="FHF" name="author"/>\n  <meta content="eZ Platform" name="generator"/>\n  <title>\n   Le dico médical - hopital.fr - Fédération Hospitalière de France\n  </title>\n  <link href="/Le-dico-medical" rel="canonical"/>\n  <link href="/bundles/fhfhopital/images/favicon.ico?1746072309" rel="Shortcut icon" type="image/x-icon"/>\n  <meta content="width=device-width, initial-scale=1, minimum-scale=1" name="viewport"/>\n  <meta content="r1dfTyNDZJ8yTfi353Pxx1HwLE_1VHbYOrR4wNlxPJc" name="google-site-verification"/>\n  <link crossorigin="" href="https://fonts.gstatic.com/" rel="preconnect"/>\n  <link href="//fonts.googleapis.com" rel="dns-prefetch"/>\n  <link href="/css/948d522.css?1746072309?family=Ubuntu:300,400,500,700,300italic,400italic&amp;subset=latin,latin-ext" lazyload="" rel="stylesheet" type="text/css">\n   <link href="/css/e2d33c1.css?1746072309" rel="stylesheet">\

In [15]:
liste_vocab_medical = []

In [27]:
import requests
from bs4 import BeautifulSoup
import string
import time

url = "https://www.hopital.fr/Le-dico-medical/Les-pathologies-et-symptomes"
response = requests.get(url)
response.raise_for_status()
soup = BeautifulSoup(response.text, "html.parser")

liste_vocab_medical = []

for letter in string.ascii_lowercase:
    section = soup.find("section", class_=f"letter-block-{letter}")
    if not section:
        print(f"❌ Aucune section pour la lettre {letter.upper()}")
        continue

    ul = section.find("ul", class_="nav n2")
    if ul:
        mots = [a.text.strip().lower() for a in ul.find_all("a")]
        liste_vocab_medical.extend(mots)
    else:
        print(f"⚠️ Pas de liste trouvée pour {letter.upper()}")

    time.sleep(0.2)  # pause entre les requêtes



❌ Aucune section pour la lettre J
❌ Aucune section pour la lettre W
❌ Aucune section pour la lettre X
❌ Aucune section pour la lettre Y


In [26]:
print(f"\n✅ Total de mots récupérés : {len(liste_vocab_medical)}")
for mot in liste_vocab_medical:
    print(mot)


✅ Total de mots récupérés : 382
abcès
aboulie
accident ischémique transitoire
accident vasculaire cérébral
acidocétose
acidose
acouphène
acromégalie
addiction
adénocarcinome
adénome
adénopathie
adiposité
aérophagie
agoraphobie
algie vasculaire de la face
algodystrophie
algoneurodystrophie
allergie
alopécie
alzheimer (maladie d')
amblyopie
aménorrhée
amnésie
amyotrophie
anasarque
anémie
anévrisme
angine
angine de poitrine
angiopathie
angor
anorexie
anosmie
anthrax
apathie
aphasie
aphte
aplasie [d'un organe]
aplasie médullaire
apnée (du sommeil)
appendicite
apraxie
artérite des membres inférieurs
arthrite
arthrose
arythmie
arythmie cardiaque
ascite
asphyxie
asthme
astigmatisme
atélectasie
athérosclérose
attaque de panique
autisme
auto-immune
avc
avf
bacille de koch
bacillose
bartholinite
basedow [maladie]
bipolaire [trouble]
bipolarité
bk
blennoragie
borderline
botulisme
boulimie
bpco
bradycardie
bronchiolite
bronchite
broncho-pneumopathie chronique obstructive
brucellose
bursite
cachex

In [ ]:
# Je veux que "Rue de Robin" soit reconnu comme une localisation. Actuellement "Robin" est détecté comme étant une personne.
# On utilise donc le composant de spaCy nommé EntityRuler por spécifier et prioriser des règles d'entités.
from fastapi import FastAPI
from spacy.pipeline import EntityRuler
from pydantic import BaseModel
import httpx  # async HTTP client : on l'utilise à la place de request pour sa propriété asynchrone (= possibilité de gérer plusieurs requêtes en même temps)
import asyncio
import requests
from bs4 import BeautifulSoup

app_chatbot = FastAPI()


# Requête du client
class ChatRequest(BaseModel):
    message: str

# Réponse du bot
class ChatResponse(BaseModel):
    response: str



# On crée les patterns à identifier comme étant une localisation
ruler = nlp.add_pipe("entity_ruler", before="ner")

 
patterns = [
   
        {
            "label": "LOC",
            "pattern": [
                {"LOWER": {"IN": ["rue", "avenue", "boulevard", "place"]}},
                {"LOWER": {"IN": ["de", "d'", "du", "des"]}},
                {"IS_ALPHA": True, "OP": "+"}  # pour détecter un ou plusieurs mots alphabétiques
            ]
    },
        {
        "label": "LOC",  # Identifier comme une localisation
        "pattern": [{"SHAPE": "ddddd"}]  # Reconnaître les séquences de 5 chiffres
    }
 ]
ruler.add_patterns(patterns)








# On crée les endpoints

@app_chatbot.post("/chat", response_model = ChatResponse)
async def user_text(chat: ChatRequest):
  
 user_msg = chat.message
 spacy_doc = nlp(user_msg)

 for token in spacy_doc : 
    print(f"{token.text.lower()} → lemma: {token.lemma_.lower()}, POS: {token.pos_}")
    
 for ent in spacy_doc.ents:
    print(f"{ent.text} → label: {ent.label_}")

 lieux = [ent.text for ent in spacy_doc.ents if ent.label_ in ["GPE", "LOC", "FAC"]]
 print("LIEUX DÉTECTÉS :", lieux)

 clean_text = " ".join([
    token.lemma_.lower() for token in spacy_doc
    if not token.is_stop and not token.is_punct
])
 print("Texte nettoyé :", clean_text)

 # En attendant d'avoir réalisé l'algorithme de machine learning permettant de classer le texte de l'utilisateur comme étant "santé", 
 # "judiciaire", "maraude" ou "douche", on simule un sujet = "santé" pour la suite du text.
 sujet = "santé"
 # sujet = classifier_subject_model.predict(text)


 if len(lieux) == 1:
    print(f"Vous vous intéressé au sujet {sujet} sur la localisation {lieux[0]}")
 elif len(lieux) == 2:
    print(f"Vous vous intéressé au sujet {sujet} sur la localisation {lieux[0]}, {lieux[1]}")
 elif len(lieux) == 3:
    print(f"Vous vous intéressé au sujet {sujet} sur la localisation {lieux[0]}, {lieux[1]}, {lieux[2]}")
 elif len(lieux) == 4:
    print(f"Vous vous intéressé au sujet {sujet} sur la localisation {lieux[0]}, {lieux[1]}, {lieux[2]}, {lieux[3]}")
 else :
    print(f"Vous vous intéressé au sujet {sujet}. Pouvez-vous nous donner le code postal, s'il vous plait ?")


 return classifier(text)


# Lancer avec : uvicorn script:app --reload



IndentationError: unexpected indent (1996645456.py, line 22)

In [ ]:
# Si pas de code postal dans "lieux" alors le demander


In [ ]:
# pour la suite, je pensais trouver un moyen de matcher la / les localisations trouvées avec un code postal.
# Ce code postal pourrait nous permettre de proposer les hopitaux, commissariats, maraudes, douches (en fonction du sujet concerné) 
# dans le code postal (via requête BDD / table concernée)

# Si pas de localité ou localité insuffisante pour matcher un code postal, on pourrait proposer les deux premiers chiffres des 8 départements de L'ile de france,
# en fonction du choix réalisé, on pourrait proposer la list de CP, et en fonction du choix, la liste des hopitaux, commissariats, 
# maraudes, douches (en fonction du sujet concerné).

# Pour la liste des des CP et informations , on pourrait scrapper des sites (tel que : https://www.hopital.fr/annuaire/Paris+Ile-de-France+75/) 
# ou réaliser un fichier nous mêmes avec de fausses informations pour simuler les données ou utiliser une API disponible pour récupérer ces infos

In [67]:
# Pas sûre de garder cette partie sur TfidfVectorizer. On a trop de lignes et trop de caractères par ligne pour pouvoir voir quelque chose j pense

#from sklearn.feature_extraction.text import TfidfVectorizer

#vectorizer = TfidfVectorizer()
#X = vectorizer.fit_transform(df["clean_text"])

In [68]:
#print(vectorizer.get_feature_names_out())  # Liste des mots

In [69]:
#print(X.toarray()[10:30])  # Matrice TF-IDF sur un échantillon

In [70]:
""""
from transformers import pipeline
classifier = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")
 
result = classifier("J'adore ce produit, il est excellent !")
print(result)

"""


'"\nfrom transformers import pipeline\nclassifier = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")\n\nresult = classifier("J\'adore ce produit, il est excellent !")\nprint(result)\n\n'